# The Semantics of English Sentences

In [1]:
import nltk

## Compositional Semantics in Feature-Based Grammar

One of the guiding ideas of building logical form for a question instead of just constructing SQL query is the **Principle of Compositionality**.

Principle of Compositionality says that the meaning of a whole is defined as the meaning of its parts and of the way they are syntactically combined.

The idea is illustrated as follows:

`bark        -> <\x.bark(x)>`

`Cyril       -> <Cyril>`

`Cyril barks -> <bark(Cyril)>`

## The $\lambda$-Calculus

The mathematical notation $\left\{ w \mid w \in V \& P(w)\right\}$ denotes the set of all $w$ such that $w$ is an element of $V$ and $w$ has property $P$. We can add the **$\lambda$ operator to achieve the same effect in first-order logic. The $\lambda$ counterpart to the above statement is $\lambda w.(V(w) \wedge P(w))$

$\lambda$ is a binding operator, just as the first-order logic quantifiers are. Given the open formula $(walk(x) \wedge chew\_gum(x))$, we can bind the variable $x$ with the $\lambda$-operator to form $\lambda x.(walk(x) \wedge chew\_gum(x))$. The NLTK equivalent of this operator is `\x.(walk(x) & chew_gum(x))`.

In [2]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')
expr

<LambdaExpression \x.(walk(x) & chew_gum(x))>

In [3]:
expr.free()

set()

In [4]:
print(read_expr(r'\x.(walk(x) & chew_gum(y))'))

\x.(walk(x) & chew_gum(y))


Binding the variables in an expression is called **$\lambda$-abstraction**. Here is an example of a $\lambda$-extract: "be an $x$ such that $x$ walks and $x$ chews gum" or "have the property of walking and chewing gum". It has often been suggested that $\lambda$-abstracts are good representations for verb phrases (or subjectless clauses), particularly when arguments occur as arguments in their own right:

1. To walk and chew gum is hard
2. `hard(\x.walk(x) & chew_gum(x)))`

So the general picture is this: given an open formula $\phi$ with free variable $x$, abstracting over $x$ yields a property expression $\lambda x.\phi$ - the property of being $x$ such that $\phi$. Formally,

If $\alpha$ is of type $\tau$, and $x$ is a variable of type $e$, then `\x.`$\alpha$ is of type $\left<e, \tau\right>$. The above sentence (to walk and chew gum is hard) is a case where we say something about a property, namely saying that it is hard. However we usually attribute a property to an individual. In fact if $\phi$ is an open formula then the abstract $\lambda x.\phi$ can be used as a unary predicate. E.g. 

`\x.(walk(x) & chew_gum(x)) (gerald)` and now it says that Gerald has the property of walking and chewing gum, which has the same meaning as `(walk(gerald) & chew_gum(gerald))`.

What we have done here is remove the `\x` from the expression in the beginning and replaced all occurences of `x` with `gerald`. 

We say $\alpha \begin{bmatrix}\beta/x \end{bmatrix}$ is the operation of replacing all free occurences of $x$ in $\alpha$ with the expression $\beta$, so `(walk(x) & chew_gum(x)) [gerald/x]` is now the same expression as `(walk(gerald) & chew_gum(gerald))`. This operation is called **$\beta$-reduction** and there is such a functionality in NLTK:

In [5]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x)) (gerald)')
print(expr)

\x.(walk(x) & chew_gum(x))(gerald)


In [6]:
print(expr.simplify())

(walk(gerald) & chew_gum(gerald))


Note that earlier the body of the $\lambda$ expression is an open formula but it is not restricted to just this. It can be any open formula. Here is an example with two $\lambda$s:
`\x.\y.(dog(x) & own(y, x))` and this is an example of a binary predicate. In NLTK, the nested $\lambda$s `\x.\y.` should be written in the abbreviated form `\x y`.

In [7]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x)) (cyril)').simplify())

\y.(dog(cyril) & own(y,cyril))


In [8]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x)) (cyril, angus)').simplify())

(dog(cyril) & own(angus,cyril))


All our $\lambda$ abstracts involve the first-order variables, `x` and `y`, variables of type $e$. But suppose we want to treat one abstract, say `x.walk(x)` as the *argument* of another $\lambda$ abstract, we might try this:

`\y.y(angus)(\x.walk(x))`
but since `y` is of type $e$, `\y.y(angus)` only applies to arguments of type $e$ while `\x.walk(x)` is of type $\left<e, t\right>$. Instead we need to allow abstraction over variables of higher type.

Let's use variables `P` and `Q` as variables of type $\left<e,t\right>$ and then we can have an abstract such as `P.P(angus)`. Since `P` is of type $\left<e,t\right>$, the whole abstract is of type $\left< \left<e, t\right>, t\right>$. Then `\P.P(angus)(\x.walk(x))` is legal, and can be simplified via $\beta$-reduction to `\x.walk(x) (angus)` and then again to `walk(angus)`.

When carrying out $\beta$-reduction, come care needs to be taken with variables. Consider 

- `\y.see(y, x)`
- `\y.see(y, z)`

Suppose now we apply the $\lambda$-term `\P.exists x.P(x)` to each of these terms:

- `P.exists x.P(x)(\y.see(y, x))`
- `P.exists x.P(x)(\y.see(y, z))`

Observe if we let the free variable `x` in `\y.see(y, x)` fall inside the scope of the existential quantifier in `P.exists x.P(x)(\y.see(y, x))` then after reduction, the result will be `exists.x(see(x, x))`. Contrast this with the second expression: `exists.x(see(x, z))`. The meanings are different The first is `x` seeing himself while the second is `x` seeing some other person `z`.

Note that given any variable-binding expression($\exists$, $\lambda$ or $\forall$), the name chosen for the bound variable is completely arbitrary. `exists x.P(x)` and `exists(y).p(y)` are equivalent: they are called **$\alpha$-equivalent** or **alphabetic variants**. The process of re-labelling bound variables is **$\alpha$-conversion**. When we test for equality of `VariableBinderExpression`s in the NLTK `logic` module, we are in fact testing for $\alpha$-equivalence.

In [9]:
expr1 = read_expr('exists x.P(x)')
expr1

<ExistsExpression exists x.P(x)>

In [10]:
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))
expr2

<ExistsExpression exists z.P(z)>

In [11]:
expr1 == expr2

True

When $\beta$-reduction is carried out on an application `f(a)`, we check if there are free variables in `a` which also occur as bound variables in any subterms of `f`. If `x` is free in `a` and that `f` contains the subterm `exists x.P(x)`. In this case, we produce an alphabetic variant of `exists x.P(x)` say `exists z1.P(z1)` and then carry on with the reduction. This relabelling is automatically done in `logic`:

In [12]:
expr3 = read_expr('\P.(exists x.P(x))(\y.see(y, x))')
print(expr3)

(\P.exists x.P(x))(\y.see(y,x))


In [13]:
print(expr3.simplify())
#returns

exists z1.see(z1,x)


## Quantified NPs

Let's now add quantifiers to the mix. We would like to express the sentence 

`A dog barks` to the form `exists x.(dog(x) & bark(x))`. The key step to this is to give a semantic representation to the quantified `NPS` *a dog* so that it can be combined with `bark` to give the logical statement. First, let's make the subject's `sem` value act as the function expression rather than the argument (this is sometimes called **type-raising**). Now, we are looking for a way of instantiating `?np` so that `[SEM=<?np(\x.bark(x))>]` is equivalent to `[SEM=<exists x.(dog(x) & bark(x))>]`. This looks like carrying out $\beta$-reduction in $\lambda$-calculus. In other words, we want a $\lambda$ term $M$ to replace `?np` so that applying $M$ to `bark` yields `exists x.(dog(x) & bark(x))`. To do this, replace the occurence `bark` by a predicate variable `P` and bind the variable to $\lambda$:

- `P.exists x.(dog(x) & P(x))`

and this is of type $\left<e, \left<e,t\right>\right>$. We will take this to be tye type of `NPS` in general. To illustrate further, a universally quantified `NP` will look like:

`\P.all x.(dog(x) -> P(x))`

By combining the semantics of the determiner $\alpha$ with the semantics of `dog`, we get:

`\Q P.exists x.(Q(x) & P(x))`.

So, 

Applying `\Q P.exists x.(Q(x) & P(x))`  as a function expression to `dog` yields `P.exists x.(dog(x) & P(x))` and carrying out $\beta$-reduction yields `exists x.(dog(x) & bark(x))`

## Transistive Verbs

The next challenge is to deal with transistive verbs, like:

`Angus chases a dog.`

The output semantics we want to build is

`exists x.(dog(x) & chase(angus, x))`. Here's a semantic expression for `chases a dog`:

`\y.exists x.(dog(x) & chase(y, x))`

Think of this as the property of being a $y$ such that for some $dog$ $x$, $y$ chases $x$. More basically, $y$ chases a dog.

Let's carry out the inverse of $\beta$-reduction on the above expression giving:

`\P.exists x.(dog(x) & P(x))(\z.chase(y, z))`

It involves the quantified `NP` representation from above to `\z.chase(y,z)` and is equivalent via $\beta$-reduction to `exists x.(dog(x) & chase(y, x))`.

Let's replace the function expression by a variable `x` of the same type as an `NP`, that is of type $\left< \left<e,t\right>, t\right>$.

`X(\z.chase(y, z))`

We finally give `chases` the semantic representation:

`\X y.X(\x.chase(y, x))`

If the last expression is applied to `\P.exists x.(dog(x) & P(x))` then the result after $\beta$-reduction is equivalent to what we want.

In [14]:
read_expr = nltk.sem.Expression.fromstring
tvp = read_expr(r'\X x.X(\y.chase(x,y))')
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')
vp = nltk.sem.ApplicationExpression(tvp, np)
print(vp)

(\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x)))


In [15]:
print(vp.simplify())

\x.exists z2.(dog(z2) & chase(x,z2))


For proper nouns, we re-interpret them such that they are also function expressions, like quantified `NP`s:

`\P.P(angus)`

This denotes the characteristic function corresponding to the set of all properties which are true of Angus. Converting from an individual constant `angus` to `\P.P(angus)` is also considered type-raising, and allows us to replace a Boolean-valued application such as `\x.walk(x)(angus)` with an equivalent function application `\P.P(angus)(\x.walk(x))`. Using $\beta$-reduction, both expressions reduce to `walk(angus)`.

In [16]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
sentence = 'Angus gives a bone to every dog'
tokens = sentence.split()
for tree in parser.parse(tokens):
    print(tree.label()['SEM'])

all z4.(dog(z4) -> exists z3.(bone(z3) & give(angus,z3,z4)))


In [17]:
#Comparing result with textbook result, similar due to alphabetic variants.
#all z4.(dog(z4) -> exists z3.(bone(z3) & give(angus,z3,z4)))
#all z2.(dog(z2) -> exists z1.(bone(z1) & give(angus,z1,z2)))

In [18]:
sents = ['Irene walks', 'Cyril bites an ankle']
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
for results in nltk.interpret_sents(sents, grammar_file):
    for (synrep, semrep) in results:
        print(synrep)

(S[SEM=<walk(irene)>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(irene)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(irene)>] Irene))
  (VP[NUM='sg', SEM=<\x.walk(x)>]
    (IV[NUM='sg', SEM=<\x.walk(x)>, TNS='pres'] walks)))
(S[SEM=<exists z5.(ankle(z5) & bite(cyril,z5))>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(cyril)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(cyril)>] Cyril))
  (VP[NUM='sg', SEM=<\x.exists z5.(ankle(z5) & bite(x,z5))>]
    (TV[NUM='sg', SEM=<\X x.X(\y.bite(x,y))>, TNS='pres'] bites)
    (NP[NUM='sg', SEM=<\Q.exists x.(ankle(x) & Q(x))>]
      (Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>] an)
      (Nom[NUM='sg', SEM=<\x.ankle(x)>]
        (N[NUM='sg', SEM=<\x.ankle(x)>] ankle)))))


We have seen how to convert English sentences to logical forms, and now see how logical forms can be checked as true or false in a model. Now, we can check the truth value of English sentences in a given model. 

In [19]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
g = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
sent = 'Cyril sees every boy'
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
for (syntree, semrep, value) in results:
    print(semrep)
    print(value)

all z6.(boy(z6) -> see(cyril,z6))
True


## Quantifier Ambiguity Revisited